In [1]:
import gdown
url = 'https://app.roboflow.com/ds/Nu4o8sn8Ds?key=v4EID8IxBn' #**
output='smart.zip'
gdown.download(url, output, quiet=False)

Downloading...
From: https://app.roboflow.com/ds/Nu4o8sn8Ds?key=v4EID8IxBn
To: /content/smart.zip
100%|██████████| 1.87G/1.87G [01:17<00:00, 24.0MB/s]


'smart.zip'

In [2]:
!mkdir /content/smart

mkdir: cannot create directory ‘/content/smart’: File exists


In [3]:
import os
os.chdir('/content/')  #change dir
!unzip -u smart.zip -d smart/Visual-Pollution-1

Archive:  smart.zip


In [4]:
!nvidia-smi

Sat Jan 21 09:21:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [6]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-72-g064365d Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 31.8/166.8 GB disk)


In [7]:
# you can train new model :
# Please note that data.yaml need to be edited to fix train/validation paths before. running the train.py
#!python /content/yolov5/train.py --img 960 --batch 24 --epochs 75 --data /content/smart/Visual-Pollution-1/data.yaml --weights yolov5x.pt --cache

In [8]:
%cd /content/

/content


In [9]:
# or load trained model for inference
#Load phase three trained model (trained earlier)
url='https://docs.google.com/uc?export=download&id=1--xn75VlBCaxx2rE4IIFiyHWRTGO2R5X&confirm=t'
output='phase3_yolo5_trained.pt'
gdown.download(url, output, quiet=False) #, use_cookies=True)


Downloading...
From: https://docs.google.com/uc?export=download&id=1--xn75VlBCaxx2rE4IIFiyHWRTGO2R5X&confirm=t
To: /content/phase3_yolo5_trained.pt
100%|██████████| 173M/173M [00:00<00:00, 267MB/s]


'phase3_yolo5_trained.pt'

In [10]:
import gdown

url='https://docs.google.com/uc?export=download&id=1_lvjNOUjM_rQYHNpzcuO_QsbTqELWbMs&confirm=t'
output='orgdataset.zip'
gdown.download(url, output, quiet=False) #, use_cookies=True)


Downloading...
From: https://docs.google.com/uc?export=download&id=1_lvjNOUjM_rQYHNpzcuO_QsbTqELWbMs&confirm=t
To: /content/orgdataset.zip
100%|██████████| 2.34G/2.34G [00:06<00:00, 336MB/s]


'orgdataset.zip'

In [11]:
import os
os.chdir('/content/')  #change dir
!unzip -u orgdataset.zip -d dataset/

Archive:  orgdataset.zip


In [12]:
import pandas as pd
test_df = pd.read_csv("/content/dataset/dataset/test.csv")
test_df

,image_path
0,953ab1447c46ecfef67ab14629cd70c7.jpg
1,e4ddbaa7970fca225a51288ce5f7d3f9.jpg
2,5b8120d69607a077b7583334be3ba18b.jpg
3,138b1dc82005b4c33e4886260649d313.jpg
4,0f91ec1533b845b13089f8cf4e0a36f7.jpg
...,...
2087,3e7dd159466f14474a0173e9b02f512a.jpg
2088,a464d4600589ddf541bed59942c0228d.jpg
2089,0a490958aac3c79bb815a0f1f6825284.jpg
2090,15afe8bce6c71deed2d53f56b78b90a8.jpg


In [13]:
model = torch.hub.load('/content/yolov5', 'custom', path='/content/phase3_yolo5_trained.pt', source='local') 


YOLOv5 🚀 v7.0-72-g064365d Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 322 layers, 86233975 parameters, 0 gradients, 203.9 GFLOPs
Adding AutoShape... 


In [14]:
%cd yolov5


/content/yolov5


In [ ]:
from tqdm import tqdm

count_test = 0 

smart_dict={'GRAFFITI' : 0.0 , 'FADED_SIGNAGE': 1.0 , 'POTHOLES': 2.0,
            'GARBAGE' : 3.0 , 'CONSTRUCTION_ROAD': 4.0 , 'BROKEN_SIGNAGE': 5.0,
            'BAD_STREETLIGHT' : 6.0 , 'BAD_BILLBOARD': 7.0 , 'SAND_ON_ROAD':8.0,
            'CLUTTER_SIDEWALK' : 9.0 , 'UNKEPT_FACADE': 10.0}



sub_df = pd.DataFrame(columns=['class','image_path','name','xmax','xmin','ymax','ymin'])
for index, row in tqdm(test_df.iterrows()):  
  t_path = '/content/dataset/dataset/images/'+row["image_path"]
  outputs = model(t_path , size=(960,540)) #, augment=True) 
  pred_boxes = outputs.pandas().xyxy[0]
  
  
  if len(pred_boxes)>0:
    one_pass = False

    for idx in range(len(pred_boxes)):
      xmin, ymin, xmax, ymax,score,cls,name = pred_boxes.iloc[idx]

     
      if name == "None":
         continue
      pclass = smart_dict[name]
      image_path = row["image_path"]
      
      if score >= 0.3:
  
        xmin = xmin/2.0
        ymin = ymin/2.0
        xmax = xmax/2.0
        ymax = ymax/2.0
   


        sub_df = sub_df.append({'class':pclass, 'image_path':image_path, 'name':name, 'xmax':xmax, 'xmin':xmin, 'ymax':ymax, 'ymin':ymin}, ignore_index=True)
        one_pass = True

    if one_pass == False:
      sub_df = sub_df.append({'class':3.0, 'image_path':row["image_path"], 'name':"GARBAGE", 'xmax':1260, 'xmin':660, 'ymax':660, 'ymin':420}, ignore_index=True)
      count_test +=1
  else:
    sub_df = sub_df.append({'class':3.0, 'image_path':row["image_path"], 'name':"GARBAGE", 'xmax':1260, 'xmin':660, 'ymax':660, 'ymin':420}, ignore_index=True)
    count_test +=1

123it [00:25,  4.69it/s]

In [ ]:
sub_df

In [ ]:
sub_df.to_csv("/content/phase3_yolo_results.csv",index=False)